In [ ]:
import numpy as np
import torch 
import os
from collections import Counter, OrderedDict
import pandas as pd
from IPython.display import display, HTML
import numbers


In [ ]:
'''
Load results of all methods tested for starting task id task_num
'''

def load_task_results(task_num = 0):

    '''
    Load results of all methods tested of a task_num
    '''
    base_folders = './experiments/domain'+str(task_num)+ '/'
    raw_records = {}
    seed_folders = ['1111/'] # select the seed that you want to check

    for seed_folder in seed_folders:

        raw_records[seed_folder] = {}
        for exp_folder in sorted(os.listdir(base_folders + seed_folder)):
            if exp_folder != '.DS_Store' and exp_folder != 'exemplar_distillation_loss_250_backup':
                #print (exp_folder)
                path = base_folders + seed_folder + exp_folder + '/lm/model/' # replace to '/cvae/modeel/' if you test cvae
                raw_records[seed_folder][exp_folder] = []
                for file in sorted(os.listdir(path)):

                    if file != '.DS_Store':
                        ckp = torch.load(path+file,map_location='cpu')
                        if 'perf' not in ckp.keys():
                            continue
                        raw_records[seed_folder][exp_folder].append(ckp['perf'])
                        
        
    return raw_records # [seed][method]

In [ ]:
def tabulate_results_all(records, first=0):
    '''
    tabulate results for omega_all, omega_first
    (the first domain: 0) replace to the first of {task_seq} in c{onfig} if you use other order
    '''
    
    formated_results = []
    all_results = {}
    for seed, results in records.items():
        for method, perf in results.items():
            #print(method)
            if len(perf) < 6:
                continue

            result = []
            for run in perf:
                # all
                result.append(run['cul_se'])
                result.append(run['cul_bleu'][3])
                # base 
                result.append(run['se'][first])
                #print(run['bleu'][first][3])
                result.append(run['bleu'][first][3])              

            result = np.asarray(result).reshape(6,4).T.mean(1)
            
            if method in all_results.keys():
                all_results[method] += [result]
            else:
                all_results[method] = [result]
    
    for method, result in all_results.items():
        result = np.array(result)
        mean_result = list(result.mean(axis=0))
    
        formated_results.append( [method] + mean_result )

    headers = ['Methods', 'ALL SER', 'ALL BLEU', 'First SER','First BLEU']
    
    return formated_results, headers

In [ ]:
'''
to check overall results for a starting order
'''
task = 0  # starting task id
raw_records = load_task_results(task)
results, headers = tabulate_results_all(raw_records, task)
dataframe = pd.DataFrame(results, columns=headers)
display(HTML(dataframe.to_html()))